In [1]:
import pandas as pd

mons = pd.read_csv("/content/역별월별비율_15_22.csv")
days = pd.read_csv("/content/역별요일별비율.csv")

In [2]:
# 엑셀 파일에서 모든 시트를 읽어서 딕셔너리 형태로 저장
excel_file = pd.ExcelFile('/content/2013년 혼잡도.xlsx')
sheet_names = excel_file.sheet_names  # 시트 이름 목록
sheet_names

/usr/local/lib/python3.10/dist-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: '일요일 상선'!$1:$126.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/usr/local/lib/python3.10/dist-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: '일요일 하선'!$1:$126.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/usr/local/lib/python3.10/dist-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: '토요일 상선'!$1:$126.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/usr/local/lib/python3.10/dist-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: '토요일 하선'!$1:$126.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/usr/local/lib/python3.10/dist-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: '평일 상선'!$1:$126.
  w

['평일 상선', '평일 하선', '토요일 상선', '토요일 하선', '일요일 상선', '일요일 하선']

In [3]:
# 모든 시트의 데이터를 저장할 빈 리스트를 생성
all_dfs = []

for sheet_name in sheet_names:
    # 필요한 행부터 읽기 시작
    df = pd.read_excel(excel_file, sheet_name=sheet_name, header=2)

    # 불필요한 열 삭제
    df.dropna(axis=1, how='all', inplace=True)

    # 요일구분 컬럼 추가
    day_of_week = sheet_name.split()[-2]  # 시트 이름에서 요일 정보 추출
    df['요일구분'] = day_of_week

    # 상하구분 컬럼 추가 (상선 또는 하선)
    direction = '상선' if '상선' in sheet_name else '하선'
    df['상하구분'] = direction

    # 데이터프레임을 리스트에 추가
    all_dfs.append(df)

In [4]:
all_dfs

[      호선     역명  방향  5시대  6시대  7시대  8시대  9시대  10시대  11시대  ...  17시대  18시대  \
 0    1호선    서울역  상선   10   20   43   72   54    35    42  ...    33    54   
 1    NaN     시청  상선   10   20   40   66   55    39    47  ...    41    72   
 2    NaN     종각  상선   10   17   29   50   46    35    44  ...    45    79   
 3    NaN   종로3가  상선   10   17   31   57   50    42    47  ...    54    96   
 4    NaN   종로5가  상선   10   16   26   45   42    37    43  ...    57    97   
 ..   ...    ...  ..  ...  ...  ...  ...  ...   ...   ...  ...   ...   ...   
 118  NaN     이촌  상선   16   21   44   72   42    35    37  ...    40    44   
 119  NaN     동작  상선   17   23   47   69   45    31    40  ...    42    44   
 120  NaN  총신대입구  상선   18   26   51   75   42    32    40  ...    40    45   
 121  NaN     사당  상선   19   24   51   70   42    32    38  ...    42    46   
 122  NaN    남태령  상선   24   37  107  138   89    60    65  ...    71    95   
 
      19시대  20시대  21시대  22시대  23시대  24시대  요일구분  상하구분  
 0     

In [5]:
# 리스트에 있는 모든 데이터프레임을 합치기
df_13 = pd.concat(all_dfs, ignore_index=True)

In [6]:
df_13.info

<bound method DataFrame.info of       호선     역명  방향  5시대  6시대  7시대  8시대  9시대  10시대  11시대  ...  17시대  18시대  \
0    1호선    서울역  상선   10   20   43   72   54    35    42  ...    33    54   
1    NaN     시청  상선   10   20   40   66   55    39    47  ...    41    72   
2    NaN     종각  상선   10   17   29   50   46    35    44  ...    45    79   
3    NaN   종로3가  상선   10   17   31   57   50    42    47  ...    54    96   
4    NaN   종로5가  상선   10   16   26   45   42    37    43  ...    57    97   
..   ...    ...  ..  ...  ...  ...  ...  ...   ...   ...  ...   ...   ...   
733  NaN     이촌  하선   30   11    8   14   24    26    22  ...    43    40   
734  NaN     동작  하선    0   13    9   14   25    28    24  ...    45    41   
735  NaN  총신대입구  하선    0   15   10   16   28    32    27  ...    41    39   
736  NaN     사당  하선   12   11    9   19   32    44    30  ...    35    34   
737  NaN    남태령  하선   12   22   17   31   52    89    55  ...    71    73   

     19시대  20시대  21시대  22시대  23시대  24시대  요일

In [7]:
# 시간대 컬럼 이름 변경
new_columns = {
    '5시대': '05시~06시', '6시대': '06시~07시', '7시대': '07시~08시', '8시대': '08시~09시', '9시대': '09시~10시',
    '10시대': '10시~11시', '11시대': '11시~12시', '12시대': '12시~13시', '13시대': '13시~14시', '14시대': '14시~15시',
    '15시대': '15시~16시', '16시대': '16시~17시', '17시대': '17시~18시', '18시대': '18시~19시', '19시대': '19시~20시',
    '20시대': '20시~21시', '21시대': '21시~22시', '22시대': '22시~23시', '23시대': '23시~24시', '24시대': '24시~01시'
}

df_13.rename(columns=new_columns, inplace=True)

In [8]:
df_13.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 738 entries, 0 to 737
Data columns (total 25 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   호선       24 non-null     object 
 1   역명       738 non-null    object 
 2   방향       738 non-null    object 
 3   05시~06시  738 non-null    int64  
 4   06시~07시  738 non-null    int64  
 5   07시~08시  738 non-null    int64  
 6   08시~09시  738 non-null    int64  
 7   09시~10시  738 non-null    int64  
 8   10시~11시  738 non-null    int64  
 9   11시~12시  738 non-null    int64  
 10  12시~13시  738 non-null    int64  
 11  13시~14시  738 non-null    int64  
 12  14시~15시  738 non-null    int64  
 13  15시~16시  738 non-null    int64  
 14  16시~17시  738 non-null    int64  
 15  17시~18시  738 non-null    int64  
 16  18시~19시  738 non-null    int64  
 17  19시~20시  738 non-null    int64  
 18  20시~21시  738 non-null    int64  
 19  21시~22시  738 non-null    int64  
 20  22시~23시  738 non-null    int64  
 21  23시~24시  738 non

In [9]:
# 컬럼 순서 변경
new_column_order = ['요일구분', '호선', '역명', '상하구분', '05시~06시', '06시~07시', '07시~08시', '08시~09시', '09시~10시', '10시~11시', '11시~12시',
                    '12시~13시', '13시~14시', '14시~15시', '15시~16시', '16시~17시', '17시~18시', '18시~19시', '19시~20시', '20시~21시',
                    '21시~22시', '22시~23시', '23시~24시', '24시~01시']

In [10]:
df_13 = df_13[new_column_order]

In [11]:
df_13

,요일구분,호선,역명,상하구분,05시~06시,06시~07시,07시~08시,08시~09시,09시~10시,10시~11시,...,15시~16시,16시~17시,17시~18시,18시~19시,19시~20시,20시~21시,21시~22시,22시~23시,23시~24시,24시~01시
0,평일,1호선,서울역,상선,10,20,43,72,54,35,...,33,34,33,54,42,25,29,26,15,4.0
1,평일,NaN,시청,상선,10,20,40,66,55,39,...,40,43,41,72,58,34,40,37,21,7.0
2,평일,NaN,종각,상선,10,17,29,50,46,35,...,44,45,45,79,63,40,53,51,27,6.0
3,평일,NaN,종로3가,상선,10,17,31,57,50,42,...,55,54,54,96,78,49,63,61,34,8.0
4,평일,NaN,종로5가,상선,10,16,26,45,42,37,...,53,53,57,97,82,52,64,65,35,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733,일요일,NaN,이촌,하선,30,11,8,14,24,26,...,40,41,43,40,35,32,36,34,25,0.0
734,일요일,NaN,동작,하선,0,13,9,14,25,28,...,42,42,45,41,37,35,36,35,27,0.0
735,일요일,NaN,총신대입구,하선,0,15,10,16,28,32,...,40,40,41,39,34,33,34,33,25,0.0
736,일요일,NaN,사당,하선,12,11,9,19,32,44,...,37,36,35,34,32,30,33,30,22,7.0


In [12]:
for index, row in df_13.iterrows():
    if pd.isnull(row['호선']):  # 호선 값이 NULL인 경우
        df_13.at[index, '호선'] = df_13.at[index - 1, '호선']  # 직전 행의 호선 값 복사
        df_13.at[index, '역명'] = df_13.at[index - 1, '역명']  # 직전 행의 역명 값 복사

In [13]:
# 호선 정수형으로 형식 바꾸기
df_13['호선'] = df_13['호선'].str.extract('(\d+)').astype('int64')

# 시간대 실수형으로 형식 바꾸기
time_columns = ['05시~06시', '06시~07시', '07시~08시', '08시~09시', '09시~10시',
                '10시~11시', '11시~12시', '12시~13시', '13시~14시', '14시~15시',
                '15시~16시', '16시~17시', '17시~18시', '18시~19시', '19시~20시',
                '20시~21시', '21시~22시', '22시~23시', '23시~24시', '24시~01시']

for col in time_columns:
    df_13[col] = df_13[col].astype('float64')

<ipython-input-13-2aee1187fd39>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_13['호선'] = df_13['호선'].str.extract('(\d+)').astype('int64')
<ipython-input-13-2aee1187fd39>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_13[col] = df_13[col].astype('float64')


In [14]:
df_13.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 738 entries, 0 to 737
Data columns (total 24 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   요일구분     738 non-null    object 
 1   호선       738 non-null    int64  
 2   역명       738 non-null    object 
 3   상하구분     738 non-null    object 
 4   05시~06시  738 non-null    float64
 5   06시~07시  738 non-null    float64
 6   07시~08시  738 non-null    float64
 7   08시~09시  738 non-null    float64
 8   09시~10시  738 non-null    float64
 9   10시~11시  738 non-null    float64
 10  11시~12시  738 non-null    float64
 11  12시~13시  738 non-null    float64
 12  13시~14시  738 non-null    float64
 13  14시~15시  738 non-null    float64
 14  15시~16시  738 non-null    float64
 15  16시~17시  738 non-null    float64
 16  17시~18시  738 non-null    float64
 17  18시~19시  738 non-null    float64
 18  19시~20시  738 non-null    float64
 19  20시~21시  738 non-null    float64
 20  21시~22시  738 non-null    float64
 21  22시~23시  738 non

### 혼잡도 구하기

In [22]:
melt_13 = pd.melt(df_13, id_vars=['요일구분', '호선', '역명', '상하구분'], var_name='시간대', value_name='값')
melt_13

,요일구분,호선,역명,상하구분,시간대,값
0,평일,1,서울역,상선,05시~06시,10.0
1,평일,1,서울역,상선,05시~06시,10.0
2,평일,1,서울역,상선,05시~06시,10.0
3,평일,1,서울역,상선,05시~06시,10.0
4,평일,1,서울역,상선,05시~06시,10.0
...,...,...,...,...,...,...
14755,일요일,4,당고개,하선,24시~01시,0.0
14756,일요일,4,당고개,하선,24시~01시,0.0
14757,일요일,4,당고개,하선,24시~01시,0.0
14758,일요일,4,당고개,하선,24시~01시,7.0


In [23]:
mons = mons.replace({'호선' : '1호선'}, '1')
mons = mons.replace({'호선' : '2호선'}, '2')
mons = mons.replace({'호선' : '3호선'}, '3')
mons = mons.replace({'호선' : '4호선'}, '4')

In [24]:
mons.rename(columns={'호선명': '호선', '지하철역': '역명'}, inplace=True)
mons

,호선,역명,사용월,비율
0,1,동대문,1,0.954919
1,1,동대문,2,0.882286
2,1,동대문,3,1.052501
3,1,동대문,4,1.032322
4,1,동대문,5,1.080913
...,...,...,...,...
4111,8호선,천호(풍납토성),8,0.975104
4112,8호선,천호(풍납토성),9,0.951714
4113,8호선,천호(풍납토성),10,1.020743
4114,8호선,천호(풍납토성),11,1.038566


In [25]:
melt_13['호선'] = melt_13['호선'].astype(str)
merged_df = melt_13.merge(mons, on=['호선', '역명'], how='inner')
merged_df.사용월.value_counts()
merged_df

,요일구분,호선,역명,상하구분,시간대,값,사용월,비율
0,평일,1,서울역,상선,05시~06시,10.0,1,1.002597
1,평일,1,서울역,상선,05시~06시,10.0,2,0.919665
2,평일,1,서울역,상선,05시~06시,10.0,3,1.014560
3,평일,1,서울역,상선,05시~06시,10.0,4,0.990673
4,평일,1,서울역,상선,05시~06시,10.0,5,1.034015
...,...,...,...,...,...,...,...,...
177115,일요일,4,당고개,하선,24시~01시,14.0,8,0.979430
177116,일요일,4,당고개,하선,24시~01시,14.0,9,0.977073
177117,일요일,4,당고개,하선,24시~01시,14.0,10,1.026712
177118,일요일,4,당고개,하선,24시~01시,14.0,11,1.001485


In [26]:
merged_df['월혼잡도'] = merged_df['값'] * merged_df['비율']
merged_df

,요일구분,호선,역명,상하구분,시간대,값,사용월,비율,월혼잡도
0,평일,1,서울역,상선,05시~06시,10.0,1,1.002597,10.025967
1,평일,1,서울역,상선,05시~06시,10.0,2,0.919665,9.196647
2,평일,1,서울역,상선,05시~06시,10.0,3,1.014560,10.145602
3,평일,1,서울역,상선,05시~06시,10.0,4,0.990673,9.906731
4,평일,1,서울역,상선,05시~06시,10.0,5,1.034015,10.340150
...,...,...,...,...,...,...,...,...,...
177115,일요일,4,당고개,하선,24시~01시,14.0,8,0.979430,13.712020
177116,일요일,4,당고개,하선,24시~01시,14.0,9,0.977073,13.679029
177117,일요일,4,당고개,하선,24시~01시,14.0,10,1.026712,14.373969
177118,일요일,4,당고개,하선,24시~01시,14.0,11,1.001485,14.020786


In [27]:
days.rename(columns={'요일': '요일구분'}, inplace=True)
days

,호선,역명,요일구분,비율
0,1,동대문,금,1.061534
1,1,동대문,목,1.020850
2,1,동대문,수,1.026641
3,1,동대문,월,1.005251
4,1,동대문,일,0.873151
...,...,...,...,...
1927,8,천호(풍납토성),수,1.103023
1928,8,천호(풍납토성),월,1.055937
1929,8,천호(풍납토성),일,0.622513
1930,8,천호(풍납토성),토,0.886181


In [28]:
merged_df = merged_df.replace({'요일구분' : '일요일'}, '일')
merged_df = merged_df.replace({'요일구분' : '토요일'}, '토')
merged_df.요일구분.value_counts()

평일    59040
토     59040
일     59040
Name: 요일구분, dtype: int64

In [29]:
weekday = merged_df[merged_df['요일구분'] == '평일'].copy()
# weekday
weeks = ['월', '화', '수', '목', '금']
result_df = pd.concat([weekday.assign(요일구분=요일) for 요일 in weeks], ignore_index=True)
result_df

,요일구분,호선,역명,상하구분,시간대,값,사용월,비율,월혼잡도
0,월,1,서울역,상선,05시~06시,10.0,1,1.002597,10.025967
1,월,1,서울역,상선,05시~06시,10.0,2,0.919665,9.196647
2,월,1,서울역,상선,05시~06시,10.0,3,1.014560,10.145602
3,월,1,서울역,상선,05시~06시,10.0,4,0.990673,9.906731
4,월,1,서울역,상선,05시~06시,10.0,5,1.034015,10.340150
...,...,...,...,...,...,...,...,...,...
295195,금,4,당고개,하선,24시~01시,20.0,8,0.979430,19.588599
295196,금,4,당고개,하선,24시~01시,20.0,9,0.977073,19.541470
295197,금,4,당고개,하선,24시~01시,20.0,10,1.026712,20.534241
295198,금,4,당고개,하선,24시~01시,20.0,11,1.001485,20.029694


In [30]:
fin = pd.concat([result_df, merged_df[merged_df['요일구분'].isin(['토', '일'])]], ignore_index=True)
fin.요일구분.value_counts()

월    59040
화    59040
수    59040
목    59040
금    59040
토    59040
일    59040
Name: 요일구분, dtype: int64

In [31]:
fin['호선'] = fin['호선'].astype(int)
merged_fin = fin.merge(days, on=['호선', '역명', '요일구분'], how='inner')
merged_fin

,요일구분,호선,역명,상하구분,시간대,값,사용월,비율_x,월혼잡도,비율_y
0,월,1,서울역,상선,05시~06시,10.0,1,1.002597,10.025967,1.055483
1,월,1,서울역,상선,05시~06시,10.0,2,0.919665,9.196647,1.055483
2,월,1,서울역,상선,05시~06시,10.0,3,1.014560,10.145602,1.055483
3,월,1,서울역,상선,05시~06시,10.0,4,0.990673,9.906731,1.055483
4,월,1,서울역,상선,05시~06시,10.0,5,1.034015,10.340150,1.055483
...,...,...,...,...,...,...,...,...,...,...
413275,일,4,당고개,하선,24시~01시,14.0,8,0.979430,13.712020,0.666672
413276,일,4,당고개,하선,24시~01시,14.0,9,0.977073,13.679029,0.666672
413277,일,4,당고개,하선,24시~01시,14.0,10,1.026712,14.373969,0.666672
413278,일,4,당고개,하선,24시~01시,14.0,11,1.001485,14.020786,0.666672


In [32]:
merged_fin['최종혼잡도'] = merged_fin['월혼잡도'] * merged_fin['비율_y']
merged_fin

,요일구분,호선,역명,상하구분,시간대,값,사용월,비율_x,월혼잡도,비율_y,최종혼잡도
0,월,1,서울역,상선,05시~06시,10.0,1,1.002597,10.025967,1.055483,10.582233
1,월,1,서울역,상선,05시~06시,10.0,2,0.919665,9.196647,1.055483,9.706901
2,월,1,서울역,상선,05시~06시,10.0,3,1.014560,10.145602,1.055483,10.708505
3,월,1,서울역,상선,05시~06시,10.0,4,0.990673,9.906731,1.055483,10.456381
4,월,1,서울역,상선,05시~06시,10.0,5,1.034015,10.340150,1.055483,10.913848
...,...,...,...,...,...,...,...,...,...,...,...
413275,일,4,당고개,하선,24시~01시,14.0,8,0.979430,13.712020,0.666672,9.141416
413276,일,4,당고개,하선,24시~01시,14.0,9,0.977073,13.679029,0.666672,9.119422
413277,일,4,당고개,하선,24시~01시,14.0,10,1.026712,14.373969,0.666672,9.582719
413278,일,4,당고개,하선,24시~01시,14.0,11,1.001485,14.020786,0.666672,9.347261


In [33]:
merged_fin.to_csv("df_!3_merged_fin.csv")